(Excellent) Original Kernel: https://www.kaggle.com/inversion/santa-s-2019-starter-notebook and
https://www.kaggle.com/xhlulu/santa-s-2019-4x-faster-cost-function

Explored different `cost_function` approaches to make it run fast. Using numba, evaluation of the whole prediction can be lowered as 70 us. By only evaluating the cost change of the new choice, the cost evaluation can be done in less than 10 us.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm import tqdm_notebook as tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Read in the family information and sample submission

In [ ]:
fpath = '/kaggle/input/santa-2019-workshop-scheduling/family_data.csv'
data = pd.read_csv(fpath, index_col='family_id')

fpath = '/kaggle/input/santa-2019-workshop-scheduling/sample_submission.csv'
submission = pd.read_csv(fpath, index_col='family_id')

In [ ]:
data.head()

In [ ]:
submission.head()

## Create some lookup dictionaries and define constants

You don't need to do it this way. :-)

In [ ]:
family_size_dict = data[['n_people']].to_dict()['n_people']

cols = [f'choice_{i}' for i in range(10)]
choice_dict = data[cols].to_dict()

N_DAYS = 100
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

# from 100 to 1
days = list(range(N_DAYS,0,-1))

## Original cost_function

Original cost_funciton from the starter notebook.

In [ ]:
def cost_function(prediction):

    penalty = 0

    # We'll use this to count the number of people scheduled each day
    daily_occupancy = {k:0 for k in days}
    
    # Looping over each family; d is the day for each family f
    for f, d in enumerate(prediction):

        # Using our lookup dictionaries to make simpler variable names
        n = family_size_dict[f]
        choice_0 = choice_dict['choice_0'][f]
        choice_1 = choice_dict['choice_1'][f]
        choice_2 = choice_dict['choice_2'][f]
        choice_3 = choice_dict['choice_3'][f]
        choice_4 = choice_dict['choice_4'][f]
        choice_5 = choice_dict['choice_5'][f]
        choice_6 = choice_dict['choice_6'][f]
        choice_7 = choice_dict['choice_7'][f]
        choice_8 = choice_dict['choice_8'][f]
        choice_9 = choice_dict['choice_9'][f]

        # add the family member count to the daily occupancy
        daily_occupancy[d] += n

        # Calculate the penalty for not getting top preference
        if d == choice_0:
            penalty += 0
        elif d == choice_1:
            penalty += 50
        elif d == choice_2:
            penalty += 50 + 9 * n
        elif d == choice_3:
            penalty += 100 + 9 * n
        elif d == choice_4:
            penalty += 200 + 9 * n
        elif d == choice_5:
            penalty += 200 + 18 * n
        elif d == choice_6:
            penalty += 300 + 18 * n
        elif d == choice_7:
            penalty += 300 + 36 * n
        elif d == choice_8:
            penalty += 400 + 36 * n
        elif d == choice_9:
            penalty += 500 + 36 * n + 199 * n
        else:
            penalty += 500 + 36 * n + 398 * n

    # for each date, check total occupancy
    #  (using soft constraints instead of hard constraints)
    for _, v in daily_occupancy.items():
        if (v > MAX_OCCUPANCY) or (v < MIN_OCCUPANCY):
            penalty += 100000000

    # Calculate the accounting cost
    # The first day (day 100) is treated special
    accounting_cost = (daily_occupancy[days[0]]-125.0) / 400.0 * daily_occupancy[days[0]]**(0.5)
    # using the max function because the soft constraints might allow occupancy to dip below 125
    accounting_cost = max(0, accounting_cost)
    
    # Loop over the rest of the days, keeping track of previous count
    yesterday_count = daily_occupancy[days[0]]
    for day in days[1:]:
        today_count = daily_occupancy[day]
        diff = abs(today_count - yesterday_count)
        accounting_cost += max(0, (daily_occupancy[day]-125.0) / 400.0 * daily_occupancy[day]**(0.5 + diff / 50.0))
        yesterday_count = today_count

    penalty += accounting_cost

    return penalty

In [ ]:
best = submission['assigned_day'].tolist()
cost = cost_function(best)
print(cost_function(best))

In [ ]:
%timeit cost_function(best)

## python based cost_function

In [ ]:
family_size_ls = list(family_size_dict.values())
choice_dict = data[cols].T.to_dict()
choice_dict_num = [{vv:i for i, vv in enumerate(di.values())} for di in choice_dict.values()]

# Computer penalities in a list
penalties_dict = {
    n: [
        0,
        50,
        50 + 9 * n,
        100 + 9 * n,
        200 + 9 * n,
        200 + 18 * n,
        300 + 18 * n,
        300 + 36 * n,
        400 + 36 * n,
        500 + 36 * n + 199 * n,
        500 + 36 * n + 398 * n
    ]
    for n in range(max(family_size_dict.values())+1)
} 

In [ ]:
def cost_function(prediction):
    penalty = 0

    # We'll use this to count the number of people scheduled each day
    daily_occupancy = np.zeros(N_DAYS + 1)
    
    # Looping over each family; d is the day, n is size of that family, 
    # and choice is their top choices
    for n, d, choice in zip(family_size_ls, prediction, choice_dict_num):
        # add the family member count to the daily occupancy
        daily_occupancy[d-1] += n

        # Calculate the penalty for not getting top preference
        if d not in choice:
            penalty += penalties_dict[n][-1]
        else:
            penalty += penalties_dict[n][choice[d]]

    accounting_cost = 0
    n_out_of_range = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    yesterday_count = daily_occupancy[:-1]
    today_count = daily_occupancy[1:]
    diff = np.abs(today_count - yesterday_count)
    n_out_of_range += np.sum((daily_occupancy > MAX_OCCUPANCY) | (daily_occupancy < MIN_OCCUPANCY))
    accounting_cost += np.sum(np.clip((daily_occupancy[:-1]-125)/400.0* daily_occupancy[:-1]**(0.5+diff/50.0), 0, None))
    
    penalty += accounting_cost

    return penalty

In [ ]:
assert cost == cost_function(best)

In [ ]:
%timeit cost_function(best)

## Numba based cost_function

In [ ]:
from numba import jit, prange
from numba import types, int16, int64, float32, float64

In [ ]:
choice_arr = []
for choice in choice_dict_num:
    c = [None for _ in choice]
    for k in choice:
        c[choice[k]] = k
    choice_arr.append(c)
choice_arr = np.array(choice_arr, dtype=np.int64)

family_size_arr = np.array(family_size_ls, dtype=np.int64)

penalties_arr = [None for i in range(max(penalties_dict.keys())+1)]
for k in penalties_dict:
    penalties_arr[k] = penalties_dict[k]
penalties_arr = np.array(penalties_arr, dtype=np.float32)

@jit(parallel=False)
def cost_function(prediction, choice, family_size, penalties):
    penalty = 0.

    # We'll use this to count the number of people scheduled each day
    daily_occupancy = [0 for k in range(N_DAYS + 1)]
    
    # Looping over each family; d is the day, n is size of that family, 
    # and choice is their top choices
    for i in prange(5000):
        # add the family member count to the daily occupancy
        daily_occupancy[prediction[i]-1] += family_size[i]
        
        # Calculate the penalty for not getting top preference
        idx = 10
        for j in range(10):
            if prediction[i] == choice[i,j]:
                idx = j
        penalty += penalties_arr[family_size[i],idx]

    accounting_cost = 0
    n_out_of_range = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        n_next = daily_occupancy[day + 1]
        n = daily_occupancy[day]
        n_out_of_range += (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        diff = abs(n - n_next)
        accounting_cost += max(0, (n-125.0) / 400.0 * n**(0.5 + diff / 50.0))

    penalty += accounting_cost

    return penalty

In [ ]:
best = np.array(best)
assert cost == cost_function(best, choice_arr, family_size_arr, penalties_arr)

In [ ]:
%timeit cost_function(best, choice_arr, family_size_arr, penalties_arr)

## compute cost change only

In [ ]:
def compute_cost_change(prediction, daily_occupancy, family_id, new_choice):
    penalty = 0
    
    # old penalty
    size = family_size_ls[family_id]
    old_choice = prediction[family_id]
    choice = choice_dict_num[family_id]
    if old_choice not in choice:
        penalty -= penalties_dict[size][-1]
    else:
        penalty -= penalties_dict[size][choice[old_choice]]
        
    # new penalty
    if new_choice not in choice:
        penalty += penalties_dict[size][-1]
    else:
        penalty += penalties_dict[size][choice[new_choice]]

    old_accounting_cost = 0
    n_out_of_range = 0
    daily_occupancy = [_ for _ in daily_occupancy]
    daily_occupancy[-1] = daily_occupancy[-2]

    for d in set((old_choice-1, old_choice, new_choice-1, new_choice)):
        if d < 1: continue
        n_next = daily_occupancy[d]
        n = daily_occupancy[d-1]
        n_out_of_range -= (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        diff = abs(n - n_next)
        old_accounting_cost += max(0, (n-125.0) / 400.0 * n**(0.5 + diff / 50.0))
        
    new_accounting_cost = 0
    daily_occupancy[old_choice-1] -= size
    daily_occupancy[new_choice-1] += size
    daily_occupancy[-1] = daily_occupancy[-2]
    
    for d in list(set((old_choice-1, old_choice, new_choice-1, new_choice))):
        if d < 1: continue
        n_next = daily_occupancy[d]
        n = daily_occupancy[d-1]
        n_out_of_range += (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        diff = abs(n - n_next)
        new_accounting_cost += max(0, (n-125.0) / 400.0 * n**(0.5 + diff / 50.0))
    
    penalty += new_accounting_cost - old_accounting_cost + n_out_of_range * 1000000

    return penalty

In [ ]:
best = submission['assigned_day'].tolist()
cost0 = cost_function(best, choice_arr, family_size_arr, penalties_arr)
new = best.copy()
new[0] = 1
cost1 = cost_function(new, choice_arr, family_size_arr, penalties_arr)
diff = cost1 - cost0
print(diff)

In [ ]:
best = submission['assigned_day'].tolist()
daily_occupancy = np.zeros(N_DAYS + 1, dtype=np.int)
for i in prange(len(best)):
    # add the family member count to the daily occupancy
    daily_occupancy[best[i]-1] += family_size_arr[i]
print(compute_cost_change(best, daily_occupancy, 0, 1))
print(abs(diff - compute_cost_change(best, daily_occupancy, 0, 1)))
assert abs(diff - compute_cost_change(best, daily_occupancy, 0, 1)) < 1e-6

In [ ]:
%timeit compute_cost_change(best, daily_occupancy, 0, 1)

## compute cost change only (numba)

In [ ]:
choice_arr = []
for choice in choice_dict_num:
    c = [None for _ in choice]
    for k in choice:
        c[choice[k]] = k
    choice_arr.append(c)
choice_arr = np.array(choice_arr, dtype=np.int64)

family_size_arr = np.array(family_size_ls, dtype=np.int64)

penalties_arr = [None for i in range(max(penalties_dict.keys())+1)]
for k in penalties_dict:
    penalties_arr[k] = penalties_dict[k]
penalties_arr = np.array(penalties_arr, dtype=np.float32)

@jit(nopython=True)
def compute_cost_change(prediction, choice, family_size, penalties, daily_occupancy, family_id, new_choice):
    penalty = 0.
    
    # old penalty
    size = family_size[family_id]
    old_choice = prediction[family_id]
    
    idx0 = 10
    idx1 = 10
    for j in range(10):
        if old_choice == choice[family_id, j]:
            idx0 = j
        if new_choice == choice[family_id, j]:
            idx1 = j
    penalty += penalties[size,idx1] - penalties[size,idx0]
    
    old_accounting_cost = 0
    n_out_of_range = 0
    daily_occupancy = [_ for _ in daily_occupancy]
    daily_occupancy[-1] = daily_occupancy[-2]

    for d in set((old_choice-1, old_choice, new_choice-1, new_choice)):
        if d < 1: continue
        n_next = daily_occupancy[d]
        n = daily_occupancy[d-1]
        n_out_of_range -= (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        diff = abs(n - n_next)
        old_accounting_cost += max(0, (n-125.0) / 400.0 * n**(0.5 + diff / 50.0))
        
    new_accounting_cost = 0
    daily_occupancy[old_choice-1] -= size
    daily_occupancy[new_choice-1] += size
    daily_occupancy[-1] = daily_occupancy[-2]
    
    for d in list(set((old_choice-1, old_choice, new_choice-1, new_choice))):
        if d < 1: continue
        n_next = daily_occupancy[d]
        n = daily_occupancy[d-1]
        n_out_of_range += (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        diff = abs(n - n_next)
        new_accounting_cost += max(0, (n-125.0) / 400.0 * n**(0.5 + diff / 50.0))
    
    penalty += new_accounting_cost - old_accounting_cost + n_out_of_range * 1000000

    return penalty

In [ ]:
best = submission['assigned_day'].values
cost0 = cost_function(best, choice_arr, family_size_arr, penalties_arr)
new = best.copy()
new[0] = 1
cost1 = cost_function(new, choice_arr, family_size_arr, penalties_arr)
diff = cost1 - cost0
print(diff)

daily_occupancy = np.zeros(N_DAYS + 1, dtype=np.int)
for i in prange(len(best)):
    # add the family member count to the daily occupancy
    daily_occupancy[best[i]-1] += family_size_arr[i]
print(compute_cost_change(best, choice_arr, family_size_arr, penalties_arr, daily_occupancy, 0, 1))
assert abs(diff - compute_cost_change(best, choice_arr, family_size_arr, penalties_arr, daily_occupancy, 0, 1)) < 1e-6

In [ ]:
best = submission['assigned_day'].values
cost0 = cost_function(best, choice_arr, family_size_arr, penalties_arr)
new = best.copy()
new[0] = 99
cost1 = cost_function(new, choice_arr, family_size_arr, penalties_arr)
diff = cost1 - cost0
print(diff)

daily_occupancy = np.zeros(N_DAYS + 1, dtype=np.int)
for i in prange(len(best)):
    # add the family member count to the daily occupancy
    daily_occupancy[best[i]-1] += family_size_arr[i]
print(compute_cost_change(best, choice_arr, family_size_arr, penalties_arr, daily_occupancy, 0, 99))
assert abs(diff - compute_cost_change(best, choice_arr, family_size_arr, penalties_arr, daily_occupancy, 0, 99)) < 1e-6

In [ ]:
%timeit compute_cost_change(best, choice_arr, family_size_arr, penalties_arr, daily_occupancy, 0, 99)